In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Read your data
df = pd.read_csv('processed_mrna_zscore.csv')
X = df.iloc[:, :-2]
y = df.iloc[:, -1]


def evaluate_features(X, y, selected_features, classifier, k_folds=10):
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    weighted_accuracies = []

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Train the classifier on the selected features
        classifier.fit(X_train[selected_features], y_train)

        # Make predictions on the test set
        y_pred = classifier.predict(X_test[selected_features])

        # Calculate and store the weighted accuracy
        weighted_accuracy = accuracy_score(y_test, y_pred, sample_weight=None)
        weighted_accuracies.append(weighted_accuracy)

        # Print class-wise accuracies for each fold
        class_report = classification_report(y_test, y_pred, output_dict=True)
        print(f"\nFold {i + 1} - Class-wise Accuracies:")
        print(class_report)

        print(f"Fold {i + 1} : {weighted_accuracy}")

    # Calculate the average weighted accuracy
    average_weighted_accuracy = np.mean(weighted_accuracies)
    print("Average Weighted Accuracy:", average_weighted_accuracy)

    return average_weighted_accuracy

def feature_selection_random_forest(X, y, n_estimators=100, top_n=15, k_folds=10):
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    # Initialize RandomForestClassifier
    rf_classifier = RandomForestClassifier(n_estimators=n_estimators, n_jobs=-1)

    feature_importances = np.zeros(X.shape[1])

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit the Random Forest model
        rf_classifier.fit(X_train, y_train)

        # Accumulate feature importances
        feature_importances += rf_classifier.feature_importances_

    # Average feature importances across folds
    feature_importances /= k_folds

    # Get indices of top N features
    top_n_indices = np.argsort(feature_importances)[::-1][:top_n]

    # Print and return the top features
    top_features = X.columns[top_n_indices]
    print("Top features selected by Random Forest:")
    print(top_features)

    return top_features

# Select top features using Random Forest
selected_features_rf = feature_selection_random_forest(X, y, n_estimators=100, top_n=100, k_folds=10)

# Evaluate the selected features using RandomForestClassifier
accuracy_rf = evaluate_features(X, y, selected_features_rf, RandomForestClassifier(n_estimators=100, n_jobs=-1))
print(f"Random Forest Accuracy with selected features: {accuracy_rf}")


Top features selected by Random Forest:
Index(['ADGRG1', 'RAMP3', 'SKP2', 'CCDC170', 'ZNF670', 'SLC35C1', 'SMIM5',
       'KRTAP5-3', 'SMARCE1', 'KCNE2', 'OR8G2', 'DUOX2', 'GFRA1', 'REEP3',
       'GRHPR', 'KRT24', 'IL6ST', 'INAVA', 'AADAC', 'CA9', 'ESR1', 'AK022898',
       'RPL6', 'ACADSB', 'LAD1', 'PPP1R14B', 'B4GALT5', 'TMOD3', 'AGR3',
       'GSDMB', 'CENPA', 'FBL', 'KRTAP10-8', 'SUSD3', 'PREX1', 'KRT75',
       'POLR1E', 'FAM187B', 'PLSCR2', 'SOX11', 'NOLC1', 'RBM28', 'OR7A10',
       'AP2M1', 'DDX19A', 'RORB', 'LINC01940', 'MELK', 'SPACA7', 'ZNF646',
       'MYO3A', 'UBAP2', 'WHSC1', 'ORAI3', 'DUSP13', 'EPHA2', 'SLC30A2',
       'STXBP6', 'CENPL', 'KRT10', 'SETDB1', 'FGD3', 'EEFSEC', 'CRABP1',
       'NR2F2-AS1', 'ZNF750', 'KLHDC2', 'UCK2', 'HNRNPK', 'ANP32AP1', 'DYDC1',
       'WFDC3', 'ZCCHC12', 'ARSG', 'IPO7', 'OPN1LW', 'C10orf129', 'ADD2',
       'SETD5', 'DYM', 'NUP93', 'LINC01816', 'VAMP2', 'EPHX2', 'BROX',
       'C11orf91', 'SYT2', 'TBC1D9', 'NSD3', 'FOXD2', 'SCN10A', 'N

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 1 - Class-wise Accuracies:
{'long': {'precision': 0.8717948717948718, 'recall': 0.9941520467836257, 'f1-score': 0.9289617486338798, 'support': 171}, 'medium': {'precision': 0.5, 'recall': 0.047619047619047616, 'f1-score': 0.08695652173913042, 'support': 21}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'accuracy': 0.868020304568528, 'macro avg': {'precision': 0.45726495726495725, 'recall': 0.34725703146755776, 'f1-score': 0.3386394234576701, 'support': 197}, 'weighted avg': {'precision': 0.8100351425224521, 'recall': 0.868020304568528, 'f1-score': 0.8156271369183513, 'support': 197}}
Fold 1 : 0.868020304568528


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 2 - Class-wise Accuracies:
{'long': {'precision': 0.8724489795918368, 'recall': 1.0, 'f1-score': 0.9318801089918256, 'support': 171}, 'medium': {'precision': 1.0, 'recall': 0.047619047619047616, 'f1-score': 0.0909090909090909, 'support': 21}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'accuracy': 0.8730964467005076, 'macro avg': {'precision': 0.6241496598639457, 'recall': 0.34920634920634924, 'f1-score': 0.34092973330030546, 'support': 197}, 'weighted avg': {'precision': 0.8639024137573812, 'recall': 0.8730964467005076, 'f1-score': 0.8185816728258533, 'support': 197}}
Fold 2 : 0.8730964467005076


...

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 8 - Class-wise Accuracies:
{'long': {'precision': 0.8717948717948718, 'recall': 1.0, 'f1-score': 0.9315068493150686, 'support': 170}, 'medium': {'precision': 1.0, 'recall': 0.045454545454545456, 'f1-score': 0.08695652173913045, 'support': 22}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'accuracy': 0.8724489795918368, 'macro avg': {'precision': 0.6239316239316239, 'recall': 0.34848484848484845, 'f1-score': 0.3394877903513997, 'support': 196}, 'weighted avg': {'precision': 0.8683935112506541, 'recall': 0.8724489795918368, 'f1-score': 0.8177000401113395, 'support': 196}}
Fold 8 : 0.8724489795918368


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 9 - Class-wise Accuracies:
{'long': {'precision': 0.8673469387755102, 'recall': 1.0, 'f1-score': 0.9289617486338798, 'support': 170}, 'medium': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 22}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'accuracy': 0.8673469387755102, 'macro avg': {'precision': 0.2891156462585034, 'recall': 0.3333333333333333, 'f1-score': 0.3096539162112933, 'support': 196}, 'weighted avg': {'precision': 0.7522907122032486, 'recall': 0.8673469387755102, 'f1-score': 0.8057321289171406, 'support': 196}}
Fold 9 : 0.8673469387755102

Fold 10 - Class-wise Accuracies:
{'long': {'precision': 0.8673469387755102, 'recall': 1.0, 'f1-score': 0.9289617486338798, 'support': 170}, 'medium': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 21}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'accuracy': 0.8673469387755102, 'macro avg': {'precision': 0.2891156462585034, 'recall': 0.333333333

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, classification_report

# Read your data
df = pd.read_csv('processed_mrna_zscore.csv')
X = df.iloc[:, :-2]
y = df.iloc[:, -1]


def evaluate_features(X, y, selected_features, classifier, k_folds=10):
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    weighted_accuracies = []

    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Train the classifier on the selected features
        classifier.fit(X_train[selected_features], y_train)

        # Make predictions on the test set
        y_pred = classifier.predict(X_test[selected_features])

        # Calculate and store the weighted accuracy
        weighted_accuracy = accuracy_score(y_test, y_pred, sample_weight=None)
        weighted_accuracies.append(weighted_accuracy)

        # Print class-wise accuracies for each fold
        class_report = classification_report(y_test, y_pred, output_dict=True)
        print(f"\nFold {i + 1} - Class-wise Accuracies:")
        print(class_report)

        print(f"Fold {i + 1} : {weighted_accuracy}")

    # Calculate the average weighted accuracy
    average_weighted_accuracy = np.mean(weighted_accuracies)
    print("Average Weighted Accuracy:", average_weighted_accuracy)

    return average_weighted_accuracy

def feature_selection_select_k_best(X, y, k=15, k_folds=10):
    skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

    # Initialize SelectKBest with f_classif
    k_best = SelectKBest(score_func=f_classif, k=k)

    selected_features = []

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # Fit SelectKBest on the training data
        k_best.fit(X_train, y_train)

        # Get the indices of the selected features
        selected_indices = np.where(k_best.get_support())[0]

        # Accumulate selected features across folds
        selected_features.extend(selected_indices)

    # Get the unique selected features
    selected_features = np.unique(selected_features)

    # Print and return the top features
    top_features = X.columns[selected_features]
    print(f"Top {k} features selected by SelectKBest:")
    print(top_features)

    return top_features

# Select top features using SelectKBest
selected_features_k_best = feature_selection_select_k_best(X, y, k=100, k_folds=10)

# Evaluate the selected features using RandomForestClassifier
accuracy_k_best = evaluate_features(X, y, selected_features_k_best, RandomForestClassifier(n_estimators=100, n_jobs=-1))
print(f"SelectKBest Accuracy with selected features: {accuracy_k_best}")


Top 100 features selected by SelectKBest:
Index(['PDIA6', 'MRFAP1L1', 'PURA', 'GAL', 'CDCA5', 'AUNIP', 'CA12', 'FAM83D',
       'MELK', 'MIS18A',
       ...
       'CHST15', 'EXOC6', 'MYB', 'IDUA', 'HMGA1', 'IGFBP4', 'CEP55', 'CIRBP',
       'LMNB2', 'MCM10'],
      dtype='object', length=191)


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 1 - Class-wise Accuracies:
{'long': {'precision': 0.868020304568528, 'recall': 1.0, 'f1-score': 0.9293478260869565, 'support': 171}, 'medium': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 21}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'accuracy': 0.868020304568528, 'macro avg': {'precision': 0.2893401015228427, 'recall': 0.3333333333333333, 'f1-score': 0.30978260869565216, 'support': 197}, 'weighted avg': {'precision': 0.75345924914324, 'recall': 0.868020304568528, 'f1-score': 0.8066927830500993, 'support': 197}}
Fold 1 : 0.868020304568528


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 2 - Class-wise Accuracies:
{'long': {'precision': 0.868020304568528, 'recall': 1.0, 'f1-score': 0.9293478260869565, 'support': 171}, 'medium': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 21}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'accuracy': 0.868020304568528, 'macro avg': {'precision': 0.2893401015228427, 'recall': 0.3333333333333333, 'f1-score': 0.30978260869565216, 'support': 197}, 'weighted avg': {'precision': 0.75345924914324, 'recall': 0.868020304568528, 'f1-score': 0.8066927830500993, 'support': 197}}
Fold 2 : 0.868020304568528


...

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 8 - Class-wise Accuracies:
{'long': {'precision': 0.8673469387755102, 'recall': 1.0, 'f1-score': 0.9289617486338798, 'support': 170}, 'medium': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 22}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'accuracy': 0.8673469387755102, 'macro avg': {'precision': 0.2891156462585034, 'recall': 0.3333333333333333, 'f1-score': 0.3096539162112933, 'support': 196}, 'weighted avg': {'precision': 0.7522907122032486, 'recall': 0.8673469387755102, 'f1-score': 0.8057321289171406, 'support': 196}}
Fold 8 : 0.8673469387755102


/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Fold 9 - Class-wise Accuracies:
{'long': {'precision': 0.8673469387755102, 'recall': 1.0, 'f1-score': 0.9289617486338798, 'support': 170}, 'medium': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 22}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 4}, 'accuracy': 0.8673469387755102, 'macro avg': {'precision': 0.2891156462585034, 'recall': 0.3333333333333333, 'f1-score': 0.3096539162112933, 'support': 196}, 'weighted avg': {'precision': 0.7522907122032486, 'recall': 0.8673469387755102, 'f1-score': 0.8057321289171406, 'support': 196}}
Fold 9 : 0.8673469387755102

Fold 10 - Class-wise Accuracies:
{'long': {'precision': 0.8673469387755102, 'recall': 1.0, 'f1-score': 0.9289617486338798, 'support': 170}, 'medium': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 21}, 'short': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5}, 'accuracy': 0.8673469387755102, 'macro avg': {'precision': 0.2891156462585034, 'recall': 0.333333333

/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
